In [1]:
# Import necessary libraries
import yfinance as yf
import pandas as pd
import sqlite3
import time
import requests
from bs4 import BeautifulSoup

In [2]:
# Function to get S&P 500 tickers
def get_sp500_tickers():
    url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table containing the tickers
    table = soup.find('table', {'class': 'wikitable'})
    tickers = []
    for row in table.findAll('tr')[1:]:
        ticker = row.findAll('td')[0].text.strip()
        tickers.append(ticker)
    return tickers

# Get the S&P 500 tickers
sp500_tickers = get_sp500_tickers()
print(f"Retrieved {len(sp500_tickers)} S&P 500 tickers.")

Retrieved 503 S&P 500 tickers.


In [3]:
# Path to SQLite database
db_path = 'database/stocks_data.db'

In [ ]:

# Function to fetch and save data for a ticker
def fetch_and_save_ticker(ticker, conn):
    try:
        print(f"Fetching data for {ticker}...")
        stock_data = yf.download(ticker, start='2015-01-01', end='2022-12-31', progress=False)
        
        if stock_data.empty:
            print(f"No data found for {ticker}, skipping...")
            return
        
        # Add columns for company and ticker
        stock_data['Ticker'] = ticker
        
        # Reset index to move 'Date' from index to a column
        stock_data.reset_index(inplace=True)
        
        # Save data to the SQLite database
        stock_data.to_sql('raw_stocks', conn, if_exists='append', index=False)
        print(f"Data for {ticker} saved to SQLite database.")
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")

# Open SQLite connection
with sqlite3.connect(db_path) as conn:
    # Ensure table is created if it doesn't exist
    conn.execute("""
    CREATE TABLE IF NOT EXISTS raw_stocks (
        Date TEXT,
        Open REAL,
        High REAL,
        Low REAL,
        Close REAL,
        Adj_Close REAL,
        Volume INTEGER,
        Ticker TEXT
    )
    """)
    conn.commit()

    # Fetch and save data for each ticker
    for ticker in sp500_tickers:
        fetch_and_save_ticker(ticker, conn)
        time.sleep(1)  # Delay to avoid hitting API rate limits

print("Data fetching complete. Database updated.")